# Extracción de preguntas frecuentes y respuestas sobre inscripciones

En primer lugar, se extraerán las preguntas y respuestas del mismo modo que en el notebook anterior.

In [1]:
from bs4 import BeautifulSoup
import requests
import nlpaug.augmenter.word as naw

In [2]:
url = "https://www.uv.es/uvweb/universidad/es/estudios-grado/admision/preguntas-frecuentes-1286095200713.html"
r = requests.get(url)

In [3]:
soup = BeautifulSoup(r.text, "html.parser")

In [4]:
s = soup.find("ul", {"class": "bloque-preguntas bp1286118204456"})
lines = s.find_all("li", {"class": "pregunta"})

In [5]:
lines[0]

<li class="pregunta" id="12862344630271286118204456">
<a href="javascript:desplegar('12862344630271286118204456')" style="text-decoration:none">
<h4>¿Qué es la preinscripción?</h4></a>
<span class="fa fa-chevron-down lnktitleFaq12862344630271286118204456"></span>
<div class="bloque-contenido">
<div class="respuesta respuesta-lista"><p>Es el procedimiento de admisión que sirve para ordenar, según la nota de admisión, a estudiantes que piden plaza en un grado. Es imprescindible realizar la preinscripción para poder obtener plaza en una universidad pública.</p>
</div>
</div>
</li>

In [6]:
faq_inscripciones = {}

for line in lines:
    pregunta = line.find("h4").text
    respuesta = line.find("p").text
    faq_inscripciones[pregunta] = respuesta

In [7]:
faq_inscripciones

{'¿Qué es la preinscripción?': 'Es el procedimiento de admisión que sirve para ordenar, según la nota de admisión, a estudiantes que piden plaza en un grado. Es imprescindible realizar la preinscripción para poder obtener plaza en una universidad pública.',
 'Què és el districte universitari valencià?': 'El distrito universitario valenciano está formado por la oferta de plazas de las titulaciones de grado de las cinco universidades públicas valencianas: Universitat de València, Universitat Politècnica de València, Universitat Jaume I de Castelló, Universitat d’Alacant y Universitat Miguel Hernández de Elche.',
 '¿Cómo sé que me han dado una plaza en la Universidad?': 'Los estudiantes admitidos recibirán, en la dirección de correo electrónico que hayan indicado en la solicitud de preinscripción, un mensaje que incluirá el periodo de matrícula del estudiante e instrucciones.',
 '¿Con las PAU de la Comunitat Valenciana se puede acceder a las titulaciones universitarias del resto de comuni

## Data augmentation para obtener más frases de entrenamiento

Esta información la queremos añadir como frases de entrenamiento y respuestas del intent faq_inscripciones.

Sin embargo, cada intent debe tener un mínimo de dos frases de entrenamiento y actualmente solo se tiene una.

Para solucionarlo utilizaremos *Back Translation* una técnica de *Data Augmentation* que consiste en traducir la frase original a otro idioma y luego de vuelva al español para obtener una frase diferente con el mismo significado.

En este caso utilizaremos los idiomas inglés, alemán, francés e italiano para realizar este aumento. Con la frase original tendremos un total de cinco frases de entrenamiento para cada intent.

In [8]:
back_translation_aug_en = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-en",
    to_model_name="Helsinki-NLP/opus-mt-en-es",
)

In [9]:
back_translation_aug_de = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-de",
    to_model_name="Helsinki-NLP/opus-mt-de-es",
)

In [10]:
back_translation_aug_fr = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-fr",
    to_model_name="Helsinki-NLP/opus-mt-fr-es",
)

In [11]:
back_translation_aug_it = naw.BackTranslationAug(
    from_model_name="Helsinki-NLP/opus-mt-es-it",
    to_model_name="Helsinki-NLP/opus-mt-it-es",
)

## Cambiar estructura de los datos para utilizarlos como frases de entrenamiento y respuestas

Ahora transformaremos los datos y los guardaremos en ficheros de texto con la estructura necesaria para poder utilizarlos en el chatbot.

In [12]:
nlu = []
responses = []

for n, (pregunta, respuesta) in enumerate(faq_inscripciones.items()):
    pregunta_yml = "- intent: faq_inscripciones/" + str(n) + "\n  examples: |"
    pregunta_yml += "\n    - ".join(
        [
            "",
            pregunta,
            back_translation_aug_en.augment(pregunta),
            back_translation_aug_de.augment(pregunta),
            back_translation_aug_fr.augment(pregunta),
            back_translation_aug_it.augment(pregunta),
        ]
    )
    nlu.append(pregunta_yml)
    respuesta_yml = (
        "  utter_faq_inscripciones/" + str(n) + ':\n  - text: "' + respuesta + '"'
    )
    responses.append(respuesta_yml)

In [13]:
nlu

['- intent: faq_inscripciones/0\n  examples: |\n    - ¿Qué es la preinscripción?\n    - ¿Qué es el pre-registro?\n    - ¿Cuál es el preregistro?\n    - ¿Qué es la preinscripción?\n    - ¿Qué es la pre-inscripción?',
 '- intent: faq_inscripciones/1\n  examples: |\n    - Què és el districte universitari valencià?\n    - ¿Qué es el barrio universitari valenciaà?\n    - ¿Qué es el distrito collegeari valencià?\n    - ¿Qué es el área universitaria?\n    - ¿Qué es el distrito universitario vale la pena?',
 '- intent: faq_inscripciones/2\n  examples: |\n    - ¿Cómo sé que me han dado una plaza en la Universidad?\n    - ¿Cómo sé que me han dado un lugar en la universidad?\n    - ¿Cómo sé que me dieron un puesto en la universidad?\n    - ¿Cómo sé si he tenido un puesto en la universidad?\n    - ¿Cómo sé que me dieron un puesto en la universidad?',
 '- intent: faq_inscripciones/3\n  examples: |\n    - ¿Con las PAU de la Comunitat Valenciana se puede acceder a las titulaciones universitarias del 

In [14]:
responses

['  utter_faq_inscripciones/0:\n  - text: "Es el procedimiento de admisión que sirve para ordenar, según la nota de admisión, a estudiantes que piden plaza en un grado. Es imprescindible realizar la preinscripción para poder obtener plaza en una universidad pública."',
 '  utter_faq_inscripciones/1:\n  - text: "El distrito universitario valenciano está formado por la oferta de plazas de las titulaciones de grado de las cinco universidades públicas valencianas: Universitat de València, Universitat Politècnica de València, Universitat Jaume I de Castelló, Universitat d’Alacant y Universitat Miguel Hernández de Elche."',
 '  utter_faq_inscripciones/2:\n  - text: "Los estudiantes admitidos recibirán, en la dirección de correo electrónico que hayan indicado en la solicitud de preinscripción, un mensaje que incluirá el periodo de matrícula del estudiante e instrucciones."',
 '  utter_faq_inscripciones/3:\n  - text: "Con independencia de la comunidad autónoma donde se hayan realizado las prue

In [15]:
with open(r"..\data\internal\nlu_faq_inscripciones.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(nlu))
with open(
    r"..\data\internal\responses_faq_inscripciones.txt", "w", encoding="utf-8"
) as f:
    f.write("\n".join(responses))